# Part of Speech (POS) Tagging with Hidden Markov Models (HMMs)

In this lesson, we introduce **Hidden Markov Models (HMMs)**, which are used to model sequences (including time-series data).

**HMMs** have been successfully used on both supervised and unsupervised problems with sequence data in applications like speech recognition, bioinformatics, sign language recognition, and gesture recognition.

This lesson is closely related to the project at the end of this course, where you will use a **Hidden Markov Model (HMM)** to tag parts of speech in English sentences. This is a common pre-processing task in natural language processing.

## What are parts of speech

Parts of speech are labels typically applied to words in a sentence. There are many potential parts of speech for most languages, but for this, we will focus on only three:

<img src='img_40.png' width=700 align='center'>

Identifything these in sentences is known as **part-of-speech tagging**:

<img src='img_41.png' width=700 align='center'>

where "mary" is a noun, "had" is a verb, "a" is a determinant, "little" is an adjective, and "lamb" is a noun

## Using a LookUp Table to Assign POS - Rudimentary Model

Say you have two sentences for training data:
1. "Mary saw Jane" -> {"Mary: "noun", "saw": "verb", "Jane": "noun"}
2. "Jane saw Will" -> {"Jane: "noun", "saw": "verb", "Will": "noun"}

So, we can create a table:

| | Noun | Verb|
|:-|:-:|:-:|
|Mary|1|0|
|saw|0|2|
|Jane|2|0|
|Will|1|0|

So now, if we have a sentence "Mary saw Will", we can use the lookup table and note:
* Mary is a noun 100% of the time (in training data)
* Saw is a verb 100% of the time (in training data)
* Will is a noun 100% of the time (in training data)

So we can tag this sentence as such:
* "Mary saw Will" -> {"Mary: "noun", "saw": "verb", "Will": "noun"}


<img src='img_42.png' width=700 align='center'>

So, that worked! However, we can't reasonably expect the LookUp Table method to work all, or maybe most, of the time

Let's use a more complex sentence series now as an example:

Say you have three sentences for training data:
1. "Mary will see Jane" -> {"Mary": "noun", "will": "modal", "see": "verb", "Jane": "noun"}
2. "Will will see Mary" -> {"Will": "noun", "will": "modal", "see": "verb", "Mary": "noun"}
3. "Jane will see Will" -> {"Jane": "noun", "will": "modal", "see": "verb", "Will": "noun"}

So, we can create a table:

| | Noun | Verb| Modal|
|:-|:-:|:-:|:-:|
|Mary|3|0|0|
|see|0|3|0|
|Jane|2|0|0|
|Will|2|0|3|


Now, lets take a new sentence, "Mary will see Will", and tag it based on the LookUp Table:
* Mary is a noun 100% of the time -> Mary: noun
* will is a modal $\frac{3}{5}$ and a noun $\frac{2}{5}$ of the time -> will: modal
* see is a verb 100% of the time-> see:verb

So we can tag this sentence as such:
* "Mary will see Will" -> {"Mary: "noun", "see": "verb", "will": "modal"}

But this is a problem, because we know Will is a noun and not a modal! This LookUp Table works by assigning the most prevalent POS Tag and isn't aware of context!

<img src='img_43.png' width=700 align='center'>

### Bi-Gram LookUp Table

So, how can the LookUp table take advantage of context? We can assign frequencies instead to groups of words, namely n-grams!

| | Noun-Modal (N-M)| Modal-Verb (M-V)| Verb-Noun (V-N)|
|:-|:-:|:-:|:-:|
|mary-will|1|0|0|
|will-see|0|3|0|
|see-jane|0|0|1|
|will-will|1|0|0|
|see-mary|0|0|1|
|jane-will|1|0|0|
|see-will|0|0|1|

So now, lets take the same sentence as before, "Mary will see Will", and tag it based on the bi-gram LookUp Table:
* mary will is a noun-modal 100% of the time -> Mary: noun, will: modal
* will see is a modal-verb 100% of the time -> will: modal, see: verb
* see will is a verb-noun 100% of the time -> see: verb, will: noun

So: "Mary will see Will" -> Mary:noun, will:modal, see:verb, Will: noun

<img src='img_44.png' width=700 align='center'>

### But will bi-grams always work?

Let's change our training data a bit:
1. "Mary Jane can see Will." -> {"Mary": "noun", "Jane": "noun", "can": ,"see": "verb", "Will": "noun"}
2. "Spot will see Mary" -> {"Will": "noun", "will": "modal", "see": "verb", "Mary": "noun"}
3. "Will Jane spot Mary?"
4. "Mary will pat Spot." -> {"Jane": "noun", "will": "modal", "see": "verb", "Will": "noun"}

And let's tag the sentence: "Jane will spot Will."

| | Noun-Modal (N-M)| Modal-Verb (M-V)| Verb-Noun (V-N)|etc|
|:-|:-:|:-:|:-:|:-:|
|mary-jane|0|0|0|1|
|jane-can|0|0|0|1|
|can-see|0|0|0|1|
|see-will|0|0|0|1|
|spot-will|1|0|0|0|
|will-see|0|1|0|0|
|see-mary|0|0|1|0|
|will-jane|0|0|1|0|
|jane-spot|0|0|0|1|
|spot-mary|0|0|1|0|
|mary-will|1|0|0|0|
|will-pat|0|1|0|0|
|pat-spot|0|0|1|0|
